<a href="https://colab.research.google.com/github/SakshiMhasde/AI-driven-phishing-bot/blob/main/dorkingphishy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# Function to extract emails from a web page and ensure no duplicates
def extract_emails(page_content):
    # Regex pattern for matching emails
    email_pattern = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'
    # Find all emails using regex and return as a list of unique emails (set)
    return re.findall(email_pattern, page_content)

# Function to fetch a webpage content
def fetch_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

# Function to get all Google Dorked pages for the given website
def get_indexed_pages(website):
    query = f'site:{website}'
    google_search_url = f"https://www.google.com/search?q={query}&num=100"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Fetch the Google search results page
    search_results = fetch_page(google_search_url)

    if search_results:
        soup = BeautifulSoup(search_results, 'html.parser')
        # Extract links to the indexed pages
        links = []
        for link in soup.find_all('a', href=True):
            href = link['href']
            if website in href and 'webcache' not in href:
                # Extract URL from the 'href' attribute (clean it)
                actual_link = href.split('&')[0].replace("/url?q=", "")
                links.append(actual_link)
        return links
    return []

# Main function to extract emails from all indexed pages
def extract_emails_from_site(website):
    print(f"Finding indexed pages for: {website}")
    pages = get_indexed_pages(website)

    all_emails = set()  # To store unique emails
    email_data = []     # To store emails along with the page source
    sr_no = 1           # Serial number starting from 1
    if pages:
        print(f"Found {len(pages)} indexed pages. Extracting emails...")
        for page in pages:
            content = fetch_page(page)
            if content:
                emails = extract_emails(content)
                for email in emails:
                    if email not in all_emails:  # Only add new emails
                        all_emails.add(email)    # Ensure it's unique
                        email_data.append({'Sr No': sr_no, 'Email': email, 'Page URL': page})
                        sr_no += 1               # Increment the serial number
    else:
        print("No indexed pages found.")

    return all_emails, email_data

# Input: Website to scrape
website = input("Enter the website (e.g., vitbhopal.ac.in): ").strip()

# Extract emails from the given website
emails, email_data = extract_emails_from_site(website)

if emails:
    # Save the emails to an Excel file with sequential numbering
    df = pd.DataFrame(email_data)
    output_file = "extracted_emails_sequential.xlsx"
    df.to_excel(output_file, index=False)
    print(f"\nEmails have been saved to {output_file}")
else:
    print("No emails found.")


Enter the website (e.g., vitbhopal.ac.in): vitbhopal.ac.in
Finding indexed pages for: vitbhopal.ac.in
Found 116 indexed pages. Extracting emails...
Error fetching /search?q=site:vitbhopal.ac.in: No connection adapters were found for '/search?q=site:vitbhopal.ac.in'
Error fetching /search?q=site:vitbhopal.ac.in: No connection adapters were found for '/search?q=site:vitbhopal.ac.in'
Error fetching /search%3Fq%3Dsite:vitbhopal.ac.in%26num%3D100%26sca_esv%3D923792fc01f6d6a7%26ie%3DUTF-8%26tbm%3Dshop%26source%3Dlnms%26ved%3D1t:200713%26ictx%3D111: No connection adapters were found for '/search%3Fq%3Dsite:vitbhopal.ac.in%26num%3D100%26sca_esv%3D923792fc01f6d6a7%26ie%3DUTF-8%26tbm%3Dshop%26source%3Dlnms%26ved%3D1t:200713%26ictx%3D111'
Error fetching /search?q=site:vitbhopal.ac.in: No connection adapters were found for '/search?q=site:vitbhopal.ac.in'
Error fetching /search?q=site:vitbhopal.ac.in: No connection adapters were found for '/search?q=site:vitbhopal.ac.in'
Error fetching /search?q=s